In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

# Split dataset ▶ Custom dataset

In [ ]:
import zipfile

with zipfile.ZipFile('/content/data_ver2.zip', 'r') as zip_ref:
    zip_ref.extractall('origin_dataset')

In [ ]:
import os

# tạo thư mục gốc
os.mkdir("custom_dataset")

# tạo thư mục images
os.mkdir("custom_dataset/images")
os.mkdir("custom_dataset/images/train")
os.mkdir("custom_dataset/images/val")

# tạo thư mục label
os.mkdir("custom_dataset/labels")
os.mkdir("custom_dataset/labels/train")
os.mkdir("custom_dataset/labels/val")

In [ ]:
import re
import shutil
import random

def split_items(start, des, train_ratio):

  img_st = os.path.join(start, "images/train")
  label_st = os.path.join(start, "labels")

  train_img = os.path.join(des, "images/train")
  val_img = os.path.join(des, "images/val")

  train_label = os.path.join(des, "labels/train")
  val_label = os.path.join(des, "labels/val")

  images_file = [f for f in os.listdir(img_st) if f.endswith(".png")]
  paired_files = []

  for img_file in images_file:
    name, _ = os.path.splitext(img_file)
    label_file = name + ".txt"
    label_path = os.path.join(label_st, label_file)
    if os.path.isfile(label_path):
      paired_files.append((img_file, label_file))

  random.shuffle(paired_files)

  train_count = int(len(paired_files) * train_ratio)
  train_set = paired_files[:train_count]
  val_set = paired_files[train_count:]

  def copy_pairs(pairs, img_dst, lbl_dst):
    for img_file, lbl_file in pairs:
      shutil.copy(os.path.join(img_st, img_file), os.path.join(img_dst, img_file))
      shutil.copy(os.path.join(label_st, lbl_file), os.path.join(lbl_dst, lbl_file))

  copy_pairs(train_set, train_img, train_label)
  copy_pairs(val_set, val_img, val_label)

  print(f"✅ Train: {len(train_set)} | Val: {len(val_set)}")

In [ ]:
split_items("/content/origin_dataset", "/content/custom_dataset", train_ratio=0.9)

✅ Train: 895 | Val: 100


Tạo file yaml

In [ ]:
#!rm mydataset.yaml
!echo 'train: /content/custom_dataset/images/train' >> mydataset.yaml
!echo 'val: /content/custom_dataset/images/val' >> mydataset.yaml
!echo 'nc: 4' >> mydataset.yaml
!echo "names: ['xe_may','xe_oto','xe_bus','xe_tai']" >> mydataset.yaml

# Yolov10

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov10m.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 32.1M/32.1M [00:00<00:00, 288MB/s]


In [ ]:
model.train(
    data = '/content/mydataset.yaml',
    epochs = 180,
    batch = 16,
    imgsz = 512,
    name = "prosupervjp",
)

Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/mydataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=180, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=prosupervjp, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrai

100%|██████████| 755k/755k [00:00<00:00, 21.4MB/s]

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1     78720  ultralytics.nn.modules.block.SCDown          [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1    228672  ultralytics

 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  2    517632  ultralytics.nn.modules.block.C2f             [576, 192, 2]                 
 17                  -1  1    332160  ultralytics.nn.modules.conv.Conv             [192, 192, 3, 2]              
 18            [-1, 13]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 19                  -1  2    831744  ultralytics.nn.modules.block.C2fCIB          [576, 384, 2, True]           
 20                  -1  1    152448  ultralytics.nn.modules.block.SCDown          [384, 384, 3, 2]              
 21            [-1, 10]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 22                  -1  2   1911168  ultralytics.nn.modules.block.C2fCIB          [960,

100%|██████████| 5.35M/5.35M [00:00<00:00, 109MB/s]


...

                   all        100       1533      0.866      0.885      0.921      0.782



180 epochs completed in 1.284 hours.
Optimizer stripped from runs/detect/prosupervjp/weights/last.pt, 33.5MB
Optimizer stripped from runs/detect/prosupervjp/weights/best.pt, 33.5MB

Validating runs/detect/prosupervjp/weights/best.pt...
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10m summary (fused): 136 layers, 15,315,484 parameters, 0 gradients, 58.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.66it/s]


                   all        100       1533      0.912      0.883      0.943      0.797
                xe_may        100       1066      0.885      0.913      0.946      0.709
                xe_oto         96        358      0.899       0.93      0.966      0.844
                xe_bus         19         20      0.943      0.834      0.936      0.836
                xe_tai         54         89      0.922      0.854      0.924        0.8
Speed: 0.2ms preprocess, 7.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/prosupervjp


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aab7d77fb90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0